In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [2]:
# Load the data
cost_matrix_df = pd.read_csv('/Users/mahinbindra/Downloads/cost_matrix.csv')
distributions_df = pd.read_csv('/Users/mahinbindra/Downloads/distributions.csv')

In [3]:
# Convert cost matrix to a NumPy array for easier access
cost_matrix = cost_matrix_df.to_numpy()

In [4]:
# Constants
ordering_cost = 24.44
num_scenarios = 1000  # Number of Monte Carlo simulations
np.random.seed(0)  # For reproducibility

In [5]:
# Generate demand scenarios using Monte Carlo simulation
demand_scenarios = np.array([
    np.random.binomial(n=row['n'], p=row['p'], size=num_scenarios)
    for index, row in distributions_df.iterrows()
])

In [6]:
# Initialize Gurobi model
model = gp.Model("Two_Stage_Stochastic_Baffin_Bay_Retail_Network")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [7]:
# First stage decision variables: initial order quantities
y = model.addVars(len(distributions_df), lb=0, vtype=GRB.CONTINUOUS, name="order_qty")

In [8]:
# Second stage decision variables: transshipment quantities for each scenario
x = model.addVars(len(distributions_df), len(distributions_df), num_scenarios, lb=0, vtype=GRB.CONTINUOUS, name="transshipment")

In [9]:
# Objective function: Minimize total cost
total_cost = ordering_cost * gp.quicksum(y[i] for i in range(len(distributions_df))) + \
             gp.quicksum(cost_matrix[i, j] * x[i, j, k] for i in range(len(distributions_df)) 
                         for j in range(len(distributions_df)) if i != j for k in range(num_scenarios))
model.setObjective(total_cost, GRB.MINIMIZE)

In [10]:
# Constraints
for k in range(num_scenarios):
    for i in range(len(distributions_df)):
        # Ensure demand is met in each scenario
        model.addConstr(
            y[i] + gp.quicksum(x[j, i, k] for j in range(len(distributions_df)) if j != i) >= demand_scenarios[i, k],
            name=f"demand_met_{i}_{k}"
        )
        # Transshipment limits based on ordered quantities and scenario demands
        for j in range(len(distributions_df)):
            if i != j:
                model.addConstr(
                    x[i, j, k] <= y[i] - demand_scenarios[i, k],
                    name=f"transship_limit_{i}_{j}_{k}"
                )

In [11]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 225000 rows, 225015 columns and 645000 nonzeros
Model fingerprint: 0x5dd5eb43
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Presolve removed 225000 rows and 225015 columns
Presolve time: 0.10s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8101960e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.15 seconds (0.24 work units)
Optimal objective  3.810196000e+04


In [12]:
# Output results
if model.status == GRB.OPTIMAL:
    print("Optimal value (total cost): $", model.ObjVal)
    print("Order quantities (y_i):")
    for i in range(len(distributions_df)):
        print(f"Retailer {i+1}: {y[i].X:.2f} units")
else:
    print("No optimal solution found.")

Optimal value (total cost): $ 38101.96000000001
Order quantities (y_i):
Retailer 1: 104.00 units
Retailer 2: 68.00 units
Retailer 3: 112.00 units
Retailer 4: 84.00 units
Retailer 5: 68.00 units
Retailer 6: 77.00 units
Retailer 7: 126.00 units
Retailer 8: 119.00 units
Retailer 9: 102.00 units
Retailer 10: 109.00 units
Retailer 11: 94.00 units
Retailer 12: 110.00 units
Retailer 13: 121.00 units
Retailer 14: 133.00 units
Retailer 15: 132.00 units


In [13]:
# Constants
selling_price = 49.99  # as given previously
ordering_cost = 24.44  # as given previously

# Assume unsold inventory cost is a certain percentage of the ordering cost
# and opportunity cost as the missed profit on lost sales
percentage_of_ordering_cost = 0.1  # this percentage should be defined based on your case
unsold_inventory_cost = percentage_of_ordering_cost * ordering_cost

# Opportunity cost is the selling price minus the cost of goods sold per unit
opportunity_cost = selling_price - ordering_cost

# Print the calculated costs
print(f"Unsold Inventory Cost per unit: ${unsold_inventory_cost:.2f}")
print(f"Opportunity Cost of Excess Inventory per unit: ${opportunity_cost:.2f}")

Unsold Inventory Cost per unit: $2.44
Opportunity Cost of Excess Inventory per unit: $25.55


In [19]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

# Load the distributions data
distributions_df = pd.read_csv('/Users/mahinbindra/Downloads/distributions.csv')

# Constants
selling_price = 49.99
ordering_cost = 24.44
unsold_inventory_cost_percentage = 0.1  # Assumed or based on your case
unsold_inventory_cost = unsold_inventory_cost_percentage * ordering_cost
opportunity_cost = selling_price - ordering_cost

# Parameters for Monte Carlo simulation
num_trials = 50
num_scenarios = 100
np.random.seed(0)  # For reproducibility

# Generate demand scenarios once to avoid re-generation in each trial
all_demand_scenarios = [
    np.random.binomial(n=row['n'], p=row['p'], size=(num_trials, num_scenarios))
    for index, row in distributions_df.iterrows()
]

def perform_trial(trial_index, demand_scenarios):
    # Initialize Gurobi model
    model = gp.Model("Retailer_Order_Quantity_Optimization")
    model.setParam('OutputFlag', 0)  # Turn off Gurobi output

    # Decision variable: order quantity for each retailer
    y = model.addVars(len(distributions_df), lb=0, vtype=GRB.CONTINUOUS, name="order_qty")

    # Calculate expected cost for this trial
    cost_expr = ordering_cost * gp.quicksum(y[i] for i in range(len(distributions_df)))  # Ordering cost

    # Iterate through each scenario in this trial to calculate the opportunity and unsold inventory costs
    for i in range(len(distributions_df)):
        scenario_costs = 0
        for k in range(num_scenarios):
            demand = demand_scenarios[i][trial_index, k]
            excess_demand = model.addVar(lb=0, ub=GRB.INFINITY, name="excess_demand_{}_{}_{}".format(i, trial_index, k))
            model.addConstr(excess_demand >= demand - y[i])
            unsold_inventory = model.addVar(lb=0, ub=GRB.INFINITY, name="unsold_inventory_{}_{}_{}".format(i, trial_index, k))
            model.addConstr(unsold_inventory >= y[i] - demand)
            
            scenario_costs += (unsold_inventory * unsold_inventory_cost + excess_demand * opportunity_cost)

        cost_expr += scenario_costs / num_scenarios

    model.setObjective(cost_expr, GRB.MINIMIZE)

    # Solve the model
    model.optimize()

    # Return the total cost for this trial
    return model.ObjVal if model.status == GRB.OPTIMAL else None

# Perform all trials and compute the average cost
total_costs = [perform_trial(trial, all_demand_scenarios) for trial in range(num_trials)]
average_cost = np.mean(total_costs)

print(f"Average cost associated with the optimal plan: ${average_cost:.2f}")


Average cost associated with the optimal plan: $33885.47


In [23]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

# Load the data
distributions_df = pd.read_csv('/Users/mahinbindra/Downloads/distributions.csv')
cost_matrix_df = pd.read_csv('/Users/mahinbindra/Downloads/cost_matrix.csv')

# Constants
ordering_cost = 24.44
unsold_inventory_cost_percentage = 0.1
unsold_inventory_cost = unsold_inventory_cost_percentage * ordering_cost
selling_price = 49.99
opportunity_cost = selling_price - ordering_cost

# Parameters for Monte Carlo simulation
num_trials = 50
num_scenarios = 100
np.random.seed(0)  # For reproducibility

# Generate demand scenarios once to avoid re-generation in each trial
all_demand_scenarios = np.array([
    np.random.binomial(n=row['n'], p=row['p'], size=(num_trials, num_scenarios))
    for index, row in distributions_df.iterrows()
])

def perform_trial(trial_index, demand_scenarios, cost_matrix):
    num_retailers = len(distributions_df)
    model = gp.Model("Retailer_Order_and_Transshipment_Optimization")
    model.setParam('OutputFlag', 0)  # Turn off Gurobi output

    y = model.addVars(num_retailers, lb=0, vtype=GRB.CONTINUOUS, name="order_qty")
    x = model.addVars(num_retailers, num_retailers, num_scenarios, lb=0, vtype=GRB.CONTINUOUS, name="transshipment_qty")

    # Calculate the total cost
    order_cost = ordering_cost * gp.quicksum(y[i] for i in range(num_retailers))
    transshipment_cost = gp.quicksum(cost_matrix.iloc[i, j] * x[i, j, k] for i in range(num_retailers) for j in range(num_retailers) if i != j for k in range(num_scenarios))

    total_cost = order_cost + transshipment_cost

    # Constraints for demand fulfillment across scenarios
    for k in range(num_scenarios):
        for i in range(num_retailers):
            demand = demand_scenarios[i][trial_index, k]
            model.addConstr(
                y[i] + gp.quicksum(x[j, i, k] for j in range(num_retailers) if j != i) 
                - gp.quicksum(x[i, j, k] for j in range(num_retailers) if i != j) >= demand,
                name=f"demand_fulfill_{i}_{k}"
            )

    model.setObjective(total_cost, GRB.MINIMIZE)

    model.optimize()

    return model.ObjVal if model.status == GRB.OPTIMAL else None

# Assuming all_demand_scenarios and cost_matrix_df are already defined
total_costs = [perform_trial(trial, all_demand_scenarios, cost_matrix_df) for trial in range(num_trials)]
average_cost = np.mean([cost for cost in total_costs if cost is not None])

print(f"Average cost associated with the optimal plan: ${average_cost:.2f}")




Average cost associated with the optimal plan: $35818.62
